In [ ]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess
import json

In [ ]:
place="singapore"

In [ ]:
# Build file path
file_path = f'./out/{place}/final/weighted_difference_between_clusters.json'

# Load JSON file
with open(file_path, 'r') as file:
    colours = json.load(file)

In [ ]:
buildings = gpd.read_parquet(f"./out/{place}/buildings.pq")
tessellation = gpd.read_parquet(f"./out/{place}/tessellation_stats_clusters.pq")
streets = gpd.read_parquet(f"./out/{place}/streets.pq")

In [ ]:
tessellation_colours = tessellation
tessellation_colours['colour_values'] = tessellation['cluster_ID'].astype(str).str.split('.').str[0].map(colours)

In [ ]:
# Map for comparison
f, ax = plt.subplots(figsize=(200, 200))
tessellation_colours.plot(column='colour_values', cmap='gist_rainbow', linewidth=0.8, ax=ax, edgecolor='0.8')
streets.plot(ax=ax, color='blue')
buildings.plot(ax=ax, color='black', alpha=0.5)
ax.set_axis_off()

# Show the plot
plt.show()

In [ ]:
# Map for comparison
f, ax = plt.subplots(figsize=(200, 200))
tessellation_colours.plot(column='cluster_ID', linewidth=0.8, ax=ax, edgecolor='0.8')
streets.plot(ax=ax, color='blue')
buildings.plot(ax=ax, color='black', alpha=0.5)
ax.set_axis_off()

# Show the plot
plt.show()

In [ ]:
file = f'./out/{place}/final/tessellation.geojson'

if os.path.exists(file):
    os.remove(file)
    print("File deleted successfully.")
else:
    print("File not found.")

In [ ]:
def set_geojson_precision_and_trim_write(n_coords: int, n_trim: int, file_path: str) -> tuple:

    with open(file_path, 'r') as f:
        data = json.load(f)
    
    def set_precision_and_trim(coords, n):
        if isinstance(coords, list):
            return [set_precision_and_trim(c, n) for c in coords]
        elif isinstance(coords, dict):
            return {k: set_precision_and_trim(v, n) for k, v in coords.items()}
        elif isinstance(coords, float):
            return round(coords, n)
        else:
            if isinstance(coords, str) and coords.isdigit():
                return coords  # Don't trim strings that are only digits
            try:
                return float(coords)  # Leave other numbers untouched
            except (ValueError, TypeError):
                return coords
    
    for i,feature in enumerate(data['features']):
        for key in data['features'][i]["properties"].keys():
            if type(data['features'][i]["properties"][key]) == float:
                data['features'][i]["properties"][key] = round(data['features'][i]["properties"][key], n_trim)
            else:
                continue
            
        try:
            data['features'][i]["geometry"]["coordinates"] = set_precision_and_trim(data['features'][i]["geometry"]["coordinates"], n_coords)
        except:
            print("test")

    # Write the modified data to a new file with the name appended with "_trimmed"
    path, ext = os.path.splitext(file_path)
    new_file_path = path + '_trimmed' + ext
    with open(new_file_path, 'w') as f:
        json.dump(data, f)
    
    return os.path.basename(new_file_path), new_file_path

In [ ]:
import contextily as ctx

In [ ]:
# Define the name and filepath of the GeoJSON file to upload

tessellation.to_crs("epsg:4326").to_file(file, driver='GeoJSON')

file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file)

In [ ]:
file = f'./out/{place}/final/buildings.geojson'

if os.path.exists(file):
    os.remove(file)
    print("File deleted successfully.")
else:
    print("File not found.")

In [ ]:
buildings.set_crs(tessellation.crs).to_crs("epsg:4326").to_file(file, driver='GeoJSON')

In [ ]:
file_name, file_path = set_geojson_precision_and_trim_write(7, 3, file)